## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 8th Oct. 15th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-10-2024,Condo/Co-op,2700 Donald Ross Rd #203,Palm Beach Gardens,FL,33410.0,4400000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11027838,N,Y,26.880648,-80.07185


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
9,PAST SALE,2024-10-11,Condo/Co-op,2105 Lavers Cir #202,Delray Beach,FL,33444.0,345000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/2105-La...,Beaches MLS,RX-11014381,N,Y,26.435281,-80.084046
10,PAST SALE,2024-10-11,Condo/Co-op,4784 S Central Blvd #27,Jupiter,FL,33458.0,420000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/4784-S-Centr...,Beaches MLS,RX-11012198,N,Y,26.889836,-80.112667
30,PAST SALE,2024-10-11,Condo/Co-op,799 Jeffery St #4040,Boca Raton,FL,33487.0,233000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/799-Jeffe...,MARMLS,A11579793,N,Y,26.403029,-80.071861
15,PAST SALE,2024-10-11,Condo/Co-op,112 Yacht Club Dr #2,North Palm Beach,FL,33408.0,315000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/112...,Beaches MLS,RX-11012069,N,Y,26.826460,-80.057750
32,PAST SALE,2024-10-11,Condo/Co-op,3301 S Ocean Blvd #706,Highland Beach,FL,33487.0,525000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3301-...,Beaches MLS,RX-10975524,N,Y,26.416101,-80.064144
35,PAST SALE,2024-10-11,Condo/Co-op,501 Sabal Ridge Cir Unit G,Palm Beach Gardens,FL,33418.0,255000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/5...,Beaches MLS,RX-10994627,N,Y,26.852863,-80.105492
36,PAST SALE,2024-10-11,Condo/Co-op,610 Clematis St #819,West Palm Beach,FL,33401.0,465000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/610-...,Beaches MLS,RX-10992232,N,Y,26.712874,-80.057620
39,PAST SALE,2024-10-11,Condo/Co-op,110 Yacht Club Way #108,Hypoluxo,FL,33462.0,243000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hypoluxo/110-Yacht-C...,Beaches MLS,RX-10986469,N,Y,26.567104,-80.052837
26,PAST SALE,2024-10-11,Condo/Co-op,6314 Chasewood Dr Unit B,Jupiter,FL,33458.0,239000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/6314-Chasewo...,Beaches MLS,RX-11005413,N,Y,26.927122,-80.132085
8,PAST SALE,2024-10-11,Condo/Co-op,3450 S Ocean Blvd #321,Palm Beach,FL,33480.0,349000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3450-S-Oc...,Beaches MLS,RX-10998041,N,Y,26.594835,-80.037303


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

57


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$387,771


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$276


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$22,102,925


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach
Input Previous Week Condo Sales Total (number units sold):
123
Input Previous Week Condo Average Sales Price:
444746
Input Previous Week Condo Average PSF:
291
Input Previous Week Condo Sales total (ex: 198_000_000)
45_900_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-10,Condo/Co-op,2700 Donald Ross Rd #203,Palm Beach Gardens,FL,33410.0,4400000.0,4.0,4.5,Azure Condo,3849.0,43560.0,2016.0,NaN,1143.0,3997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11027838,N,Y,26.880648,-80.07185,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Vincent Marotta,Illustrated Properties LLC (Co,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-10,Condo/Co-op,2700 Donald Ross Rd #203,Palm Beach Gardens,FL,33410.0,4400000.0,4.0,4.5,Azure Condo,3849.0,43560.0,2016.0,NaN,1143.0,3997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11027838,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Vincent Marotta,Illustrated Properties LLC (Co,None,None,1,orange
1,PAST SALE,2024-10-11,Condo/Co-op,3550 S Ocean Blvd Unit 6d,South Palm Beach,FL,33480.0,1905000.0,2.0,3.5,3550 South Ocean Condo,2606.0,NaN,2019.0,NaN,731.0,3109.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10947255,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Kourtney Pulitzer,"Sotheby's Intl. Realty, Inc.",Patricia Towle,"Sotheby's Intl. Realty, Inc.",Bradford Miller,Bradford P Miller Real Estate,None,None,2,blue
2,PAST SALE,2024-10-08,Condo/Co-op,20080 Boca West Dr #411,Boca Raton,FL,33434.0,1095000.0,3.0,2.5,Laurel Oaks,2200.0,NaN,1981.0,NaN,498.0,1702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beaches MLS,RX-10983353,N,Y,26.382877,-80.150595,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beth Slossberg,Boca West Realty LLC,None,None,Ted Hammers,Hammers Properties,None,None,3,blue
3,PAST SALE,2024-10-07,Condo/Co-op,1200 S Ocean Blvd Unit 10E,Boca Raton,FL,33432.0,637500.0,2.0,2.0,Cloisters Beach Tower,1230.0,NaN,1966.0,NaN,518.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Beaches MLS,F10446982,N,Y,26.332495,-80.073271,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Amy Young,"Compass Florida, LLC.",None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue
4,PAST SALE,2024-10-09,Condo/Co-op,550 Okeechobee Blvd #501,West Palm Beach,FL,33401.0,610000.0,2.0,2.0,Cityplace South Tower Condo,1441.0,NaN,2008.0,NaN,423.0,1521.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10950757,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Amanda Glass,"Sandals Realty Group, Inc.",None,None,Lynn Ciklin,Brown Harris Stevens of PB,None,None,5,blue
5,PAST SALE,2024-10-09,Condo/Co-op,3570 S Ocean Blvd #403,South Palm Beach,FL,33480.0,595000.0,2.0,2.0,Tuscany OF Palm Beach Condominium,1140.0,NaN,1969.0,NaN,522.0,1306.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Beaches MLS,RX-11027516,N,Y,26.589679,-80.038111,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,6,blue
6,PAST SALE,2024-10-11,Condo/Co-op,3301 S Ocean Blvd #706,Highland Beach,FL,33487.0,525000.0,2.0,2.0,Ambassadors V South Condo,1050.0,NaN,1975.0,NaN,500.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3301-...,Beaches MLS,RX-10975524,N,Y,26.416101,-80.064144,https://www.redfin.com/FL/Highland-Beach/3301-...,Kyle Lopez,Florida's Best Realty Services,None,None,Kyle Lopez,Florida's Best Realty Services,None,None,7,blue
7,PAST SALE,2024-10-11,Condo/Co-op,20220 Boca West Dr #1902,Boca Raton,FL,33434.0,510000.0,3.0,2.0,Cove AT Boca West Condo,1650.0,NaN,1990.0,NaN,309.0,865.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20220-Boc...,Beaches MLS,RX-11012848,N,Y,26.378322,-80.159522,https://www.redfin.com/FL/Boca-Raton/20220-Boc...,Judith Romanow,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,Judith Romanow,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,8,blue
8,PAST SALE,

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-10,Condo/Co-op,2700 Donald Ross Rd #203,Palm Beach Gardens,FL,33410.0,4400000.0,4.0,4.5,Azure Condo,3849.0,43560.0,2016.0,NaN,1143.0,3997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11027838,N,Y,26.880648,-80.07185,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Vincent Marotta,Illustrated Properties LLC (Co,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PBC_County_condo_sales_week_ending_10142024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'Azure Condo at 2700 Donald Ross Rd #203 in Palm Beach Gardens'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-10,Condo/Co-op,2700 Donald Ross Rd #203,Palm Beach Gardens,FL,33410.0,4400000.0,4.0,4.5,Azure Condo,3849.0,43560.0,2016.0,NaN,1143.0,3997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11027838,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Vincent Marotta,Illustrated Properties LLC (Co,None,None,1,orange,Azure Condo at 2700 Donald Ross Rd #203 in Pal...
1,PAST SALE,2024-10-11,Condo/Co-op,3550 S Ocean Blvd Unit 6d,South Palm Beach,FL,33480.0,1905000.0,2.0,3.5,3550 South Ocean Condo,2606.0,NaN,2019.0,NaN,731.0,3109.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10947255,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Kourtney Pulitzer,"Sotheby's Intl. Realty, Inc.",Patricia Towle,"Sotheby's Intl. Realty, Inc.",Bradford Miller,Bradford P Miller Real Estate,None,None,2,blue,3550 South Ocean Condo at 3550 S Ocean Blvd Un...
2,PAST SALE,2024-10-08,Condo/Co-op,20080 Boca West Dr #411,Boca Raton,FL,33434.0,1095000.0,3.0,2.5,Laurel Oaks,2200.0,NaN,1981.0,NaN,498.0,1702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beaches MLS,RX-10983353,N,Y,26.382877,-80.150595,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beth Slossberg,Boca West Realty LLC,None,None,Ted Hammers,Hammers Properties,None,None,3,blue,Laurel Oaks at 20080 Boca West Dr #411 in Boca...
3,PAST SALE,2024-10-07,Condo/Co-op,1200 S Ocean Blvd Unit 10E,Boca Raton,FL,33432.0,637500.0,2.0,2.0,Cloisters Beach Tower,1230.0,NaN,1966.0,NaN,518.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Beaches MLS,F10446982,N,Y,26.332495,-80.073271,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Amy Young,"Compass Florida, LLC.",None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue,Cloisters Beach Tower at 1200 S Ocean Blvd Uni...
4,PAST SALE,2024-10-09,Condo/Co-op,550 Okeechobee Blvd #501,West Palm Beach,FL,33401.0,610000.0,2.0,2.0,Cityplace South Tower Condo,1441.0,NaN,2008.0,NaN,423.0,1521.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10950757,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Amanda Glass,"Sandals Realty Group, Inc.",None,None,Lynn Ciklin,Brown Harris Stevens of PB,None,None,5,blue,Cityplace South Tower Condo at 550 Okeechobee ...
5,PAST SALE,2024-10-09,Condo/Co-op,3570 S Ocean Blvd #403,South Palm Beach,FL,33480.0,595000.0,2.0,2.0,Tuscany OF Palm Beach Condominium,1140.0,NaN,1969.0,NaN,522.0,1306.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Beaches MLS,RX-11027516,N,Y,26.589679,-80.038111,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,6,blue,Tuscany OF Palm Beach Condominium at 3570 S Oc...
6,PAST SALE,2024-10-11,Condo/Co-op,3301 S Ocean Blvd #706,Highland Beach,FL,33487.0,525000.0,2.0,2.0,Ambassadors V South Condo,1050.0,NaN,1975.0,NaN,500.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3301-...,Beaches MLS,RX-10975524,N,Y,26.416101,-80.064144,https://www.redfin.com/FL/Highland-Beach/3301-...,Kyle Lopez,Florida's Best Realty Services,None,None,Kyle Lopez,Florida's Best Realty Services,None,None,7,blue,Ambassadors V South Condo at 3301 S Ocean Blvd...
7,PAST SALE,2024-10-11,Condo/Co-op,20220 Boca West Dr #1902,Boca Raton,FL,33434.0,510000.0,3.0,2.0,Cove AT Boca West

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Azure Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $425,000M to $4,400,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Highland Beach, South Palm Beach, Lake Park, Boca Raton, Palm Beach Gardens, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 57 condo sales totaling $22,102,925 million from Oct. 8th to Oct. 15th. The previous week, brokers closed 123 condo sales totaling $45,900,000.

Last week’s units sold for an average of $387,771, lower than the $444,746 average price from the previous 

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-10,Condo/Co-op,2700 Donald Ross Rd #203,Palm Beach Gardens,FL,33410.0,4400000.0,4.0,4.5,Azure Condo,3849.0,43560.0,2016.0,NaN,1143.0,3997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11027838,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Vincent Marotta,Illustrated Properties LLC (Co,None,None,1,orange,Azure Condo at 2700 Donald Ross Rd #203 in Pal...
1,PAST SALE,2024-10-11,Condo/Co-op,3550 S Ocean Blvd Unit 6d,South Palm Beach,FL,33480.0,1905000.0,2.0,3.5,3550 South Ocean Condo,2606.0,NaN,2019.0,NaN,731.0,3109.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10947255,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Kourtney Pulitzer,"Sotheby's Intl. Realty, Inc.",Patricia Towle,"Sotheby's Intl. Realty, Inc.",Bradford Miller,Bradford P Miller Real Estate,None,None,2,blue,3550 South Ocean Condo at 3550 S Ocean Blvd Un...
2,PAST SALE,2024-10-08,Condo/Co-op,20080 Boca West Dr #411,Boca Raton,FL,33434.0,1095000.0,3.0,2.5,Laurel Oaks,2200.0,NaN,1981.0,NaN,498.0,1702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beaches MLS,RX-10983353,N,Y,26.382877,-80.150595,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beth Slossberg,Boca West Realty LLC,None,None,Ted Hammers,Hammers Properties,None,None,3,blue,Laurel Oaks at 20080 Boca West Dr #411 in Boca...
3,PAST SALE,2024-10-07,Condo/Co-op,1200 S Ocean Blvd Unit 10E,Boca Raton,FL,33432.0,637500.0,2.0,2.0,Cloisters Beach Tower,1230.0,NaN,1966.0,NaN,518.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Beaches MLS,F10446982,N,Y,26.332495,-80.073271,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Amy Young,"Compass Florida, LLC.",None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue,Cloisters Beach Tower at 1200 S Ocean Blvd Uni...
4,PAST SALE,2024-10-09,Condo/Co-op,550 Okeechobee Blvd #501,West Palm Beach,FL,33401.0,610000.0,2.0,2.0,Cityplace South Tower Condo,1441.0,NaN,2008.0,NaN,423.0,1521.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10950757,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Amanda Glass,"Sandals Realty Group, Inc.",None,None,Lynn Ciklin,Brown Harris Stevens of PB,None,None,5,blue,Cityplace South Tower Condo at 550 Okeechobee ...
5,PAST SALE,2024-10-09,Condo/Co-op,3570 S Ocean Blvd #403,South Palm Beach,FL,33480.0,595000.0,2.0,2.0,Tuscany OF Palm Beach Condominium,1140.0,NaN,1969.0,NaN,522.0,1306.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Beaches MLS,RX-11027516,N,Y,26.589679,-80.038111,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,6,blue,Tuscany OF Palm Beach Condominium at 3570 S Oc...
6,PAST SALE,2024-10-11,Condo/Co-op,3301 S Ocean Blvd #706,Highland Beach,FL,33487.0,525000.0,2.0,2.0,Ambassadors V South Condo,1050.0,NaN,1975.0,NaN,500.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3301-...,Beaches MLS,RX-10975524,N,Y,26.416101,-80.064144,https://www.redfin.com/FL/Highland-Beach/3301-...,Kyle Lopez,Florida's Best Realty Services,None,None,Kyle Lopez,Florida's Best Realty Services,None,None,7,blue,Ambassadors V South Condo at 3301 S Ocean Blvd...
7,PAST SALE,2024-10-11,Condo/Co-op,20220 Boca West Dr #1902,Boca Raton,FL,33434.0,510000.0,3.0,2.0,Cove AT Boca West

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Azure Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $425,000M to $4,400,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Highland Beach, South Palm Beach, Lake Park, Boca Raton, Palm Beach Gardens, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 57 condo sales totaling $22,102,925 million from Oct. 8th to Oct. 15th. The previous week, brokers closed 123 condo sales totaling $45,900,000.

Last week’s units sold for an average of $387,771, lower than the $444,746 average price from the previous 

In [55]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Palm-Beach-Gardens/2700-Donald-Ross-Rd-33410/unit-203/home/144739906


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/3550-S-Ocean-Blvd-33480/unit-6D/home/174328639


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/West-Palm-Beach/1060-Lake-Shore-Dr-33403/unit-201/home/42588576


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/West-Palm-Beach/610-Clematis-St-33401/unit-819/home/42613576


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach-Gardens/2700-Donald-Ross-Rd-33410/unit-203/home/144739906


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Boca-Raton/20220-Boca-West-Dr-33434/unit-1902/home/42260350


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-10,Condo/Co-op,2700 Donald Ross Rd #203,Palm Beach Gardens,FL,33410.0,4400000.0,4.0,4.5,Azure Condo,3849.0,43560.0,2016.0,NaN,1143.0,3997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Beaches MLS,RX-11027838,N,Y,26.880648,-80.071850,https://www.redfin.com/FL/Palm-Beach-Gardens/2...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Vincent Marotta,Illustrated Properties LLC (Co,None,None,1,orange,Azure Condo at 2700 Donald Ross Rd #203 in Pal...
1,PAST SALE,2024-10-11,Condo/Co-op,3550 S Ocean Blvd Unit 6d,South Palm Beach,FL,33480.0,1905000.0,2.0,3.5,3550 South Ocean Condo,2606.0,NaN,2019.0,NaN,731.0,3109.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10947255,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Kourtney Pulitzer,"Sotheby's Intl. Realty, Inc.",Patricia Towle,"Sotheby's Intl. Realty, Inc.",Bradford Miller,Bradford P Miller Real Estate,None,None,2,blue,3550 South Ocean Condo at 3550 S Ocean Blvd Un...
2,PAST SALE,2024-10-08,Condo/Co-op,20080 Boca West Dr #411,Boca Raton,FL,33434.0,1095000.0,3.0,2.5,Laurel Oaks,2200.0,NaN,1981.0,NaN,498.0,1702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beaches MLS,RX-10983353,N,Y,26.382877,-80.150595,https://www.redfin.com/FL/Boca-Raton/20080-Boc...,Beth Slossberg,Boca West Realty LLC,None,None,Ted Hammers,Hammers Properties,None,None,3,blue,Laurel Oaks at 20080 Boca West Dr #411 in Boca...
3,PAST SALE,2024-10-07,Condo/Co-op,1200 S Ocean Blvd Unit 10E,Boca Raton,FL,33432.0,637500.0,2.0,2.0,Cloisters Beach Tower,1230.0,NaN,1966.0,NaN,518.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Beaches MLS,F10446982,N,Y,26.332495,-80.073271,https://www.redfin.com/FL/Boca-Raton/1200-S-Oc...,Amy Young,"Compass Florida, LLC.",None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue,Cloisters Beach Tower at 1200 S Ocean Blvd Uni...
4,PAST SALE,2024-10-09,Condo/Co-op,550 Okeechobee Blvd #501,West Palm Beach,FL,33401.0,610000.0,2.0,2.0,Cityplace South Tower Condo,1441.0,NaN,2008.0,NaN,423.0,1521.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-10950757,N,Y,26.705124,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Amanda Glass,"Sandals Realty Group, Inc.",None,None,Lynn Ciklin,Brown Harris Stevens of PB,None,None,5,blue,Cityplace South Tower Condo at 550 Okeechobee ...
5,PAST SALE,2024-10-09,Condo/Co-op,3570 S Ocean Blvd #403,South Palm Beach,FL,33480.0,595000.0,2.0,2.0,Tuscany OF Palm Beach Condominium,1140.0,NaN,1969.0,NaN,522.0,1306.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Beaches MLS,RX-11027516,N,Y,26.589679,-80.038111,https://www.redfin.com/FL/Palm-Beach/3570-S-Oc...,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,Jeanette Romano,CGI Realty,Alexa Romano,CGI Realty,6,blue,Tuscany OF Palm Beach Condominium at 3570 S Oc...
6,PAST SALE,2024-10-11,Condo/Co-op,3301 S Ocean Blvd #706,Highland Beach,FL,33487.0,525000.0,2.0,2.0,Ambassadors V South Condo,1050.0,NaN,1975.0,NaN,500.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3301-...,Beaches MLS,RX-10975524,N,Y,26.416101,-80.064144,https://www.redfin.com/FL/Highland-Beach/3301-...,Kyle Lopez,Florida's Best Realty Services,None,None,Kyle Lopez,Florida's Best Realty Services,None,None,7,blue,Ambassadors V South Condo at 3301 S Ocean Blvd...
7,PAST SALE,2024-10-11,Condo/Co-op,20220 Boca West Dr #1902,Boca Raton,FL,33434.0,510000.0,3.0,2.0,Cove AT Boca West

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 1, 4) ## List (Earlier) date
date2 = datetime(2024, 10, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

281


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")